**By Huy Duc Vu - 25728181**

# Part 2: Business Questions (SQL only)

## 2.1

In [0]:
%sql
WITH 
base AS (
  SELECT pickup_dt,
        YEAR(pickup_dt) AS y,
        MONTH(pickup_dt) AS m,
        DATE_TRUNC('month', pickup_dt) AS ym,
        DAYOFWEEK(pickup_dt) AS dow,
        HOUR(pickup_dt) AS hr,
        passenger_count,
        total_amount
  FROM bde.final_trips_cleaned
),

monthly_aggregate AS (
  SELECT y, m, ym,
        COUNT(*) AS total_trips,
        ROUND(AVG(passenger_count), 0) AS average_passenger,
        ROUND(AVG(total_amount), 2) AS average_amount_per_trip,
        ROUND((SUM(total_amount)/SUM(passenger_count)), 2) AS average_amount_per_passenger
  FROM base
  GROUP BY y, m, ym
),

dow_ranking AS (
  SELECT y, m,
        dow,
        COUNT(*) AS dow_trip_count,
        ROW_NUMBER() OVER (PARTITION BY y,m
                           ORDER BY COUNT(*) DESC, dow) AS rn
  FROM base
  GROUP BY y, m, dow
),

busiest_dow AS (
  SELECT y, m,
    CASE dow
      WHEN 1 THEN 'Sunday'
      WHEN 2 THEN 'Monday'
      WHEN 3 THEN 'Tuesday'
      WHEN 4 THEN 'Wednesday'
      WHEN 5 THEN 'Thursday'
      WHEN 6 THEN 'Friday'
      WHEN 7 THEN 'Saturday'
    END AS busiest_dow
  FROM dow_ranking
  WHERE rn = 1
),

hr_ranking AS (
  SELECT y, m, hr,
        COUNT(*) AS hr_trip_count,
        ROW_NUMBER() OVER (PARTITION BY y, m
                           ORDER BY COUNT(*) DESC, hr) AS rn
  FROM base
  GROUP BY y, m, hr
),

busiest_hr AS (
  SELECT y, m, hr AS busiest_hr
  FROM hr_ranking
  WHERE rn = 1
)

SELECT 
  DATE_FORMAT(ym, 'yyyy-MM') AS year_month,
  total_trips,
  busiest_dow,
  busiest_hr,
  average_passenger,
  average_amount_per_trip,
  average_amount_per_passenger
FROM monthly_aggregate LEFT JOIN busiest_dow USING (y, m)
                      LEFT JOIN busiest_hr USING (y, m)
ORDER BY year_month


year_month,total_trips,busiest_dow,busiest_hr,average_passenger,average_amount_per_trip,average_amount_per_passenger
2009-01,957,Thursday,0,2.0,18.58,11.43
2011-01,2,Monday,23,1.0,12.8,12.8
2011-02,1,Tuesday,0,1.0,15.8,15.8
2014-01,14353935,Friday,19,2.0,14.4,8.69
2014-02,13844429,Saturday,19,2.0,14.53,8.83
2014-03,16461114,Saturday,19,2.0,14.69,8.95
2014-04,15696588,Wednesday,19,2.0,14.98,9.11
2014-05,15962270,Friday,19,2.0,15.55,9.45
2014-06,14928168,Sunday,19,2.0,15.56,9.45
2014-07,14170409,Thursday,19,2.0,15.21,9.24


## 2.2

In [0]:
%sql
WITH 
base AS (
  SELECT color,
        (unix_timestamp(dropoff_dt)-unix_timestamp(pickup_dt)) / 60 AS duration_min,
        trip_distance * 1.60934 AS trip_distance_km
  FROM bde.final_trips_cleaned
),

derived AS (
  SELECT color,
        duration_min,
        trip_distance_km,
        trip_distance_km / (duration_min/60) AS speed_kmh
  FROM base
)

SELECT color,
      ROUND(AVG(duration_min), 2) AS avg_duration_min,
      ROUND(percentile_approx(duration_min, 0.5), 2) AS median_duration_min,
      ROUND(MIN(duration_min), 2) AS min_duration_min,
      ROUND(MAX(duration_min), 2) AS max_duration_min,

      ROUND(AVG(trip_distance_km), 2) AS avg_distance_km,
      ROUND(percentile_approx(trip_distance_km, 0.5), 2) AS median_distance_km,
      ROUND(MIN(trip_distance_km), 2) AS min_distance_km,
      ROUND(MAX(trip_distance_km), 2) AS max_distance_km,

      ROUND(AVG(speed_kmh), 2) AS avg_speed_kmh,
      ROUND(percentile_approx(speed_kmh, 0.5), 2) AS median_speed_kmh,
      ROUND(MIN(speed_kmh), 2) AS min_speed_kmh,
      ROUND(MAX(speed_kmh), 2) AS max_speed_kmh
FROM derived
GROUP BY color
ORDER BY color;

color,avg_duration_min,median_duration_min,min_duration_min,max_duration_min,avg_distance_km,median_distance_km,min_distance_km,max_distance_km,avg_speed_kmh,median_speed_kmh,min_speed_kmh,max_speed_kmh
green,13.84,10.72,1.0,120.0,4.93,3.19,0.34,148.62,20.55,18.61,1.61,112.65
yellow,14.46,11.35,1.0,120.0,4.93,2.78,0.34,160.77,18.95,16.62,1.61,112.65


## 2.3

In [0]:
%sql
WITH 
base AS (
  SELECT color,
        PU_Borough,
        DR_Borough,
        YEAR(pickup_dt) AS y,
        MONTH(pickup_dt) AS m,
        DATE_TRUNC('month', pickup_dt) AS ym,
        DAYOFWEEK(pickup_dt) AS dow,
        HOUR(pickup_dt) AS hr,
        trip_distance * 1.60934 AS trip_distance_km, 
        total_amount
  FROM bde.final_trips_cleaned
),

derived AS (
  SELECT color,
        PU_Borough,
        DR_Borough,
        y, m, ym, dow, hr,
        COUNT(*) AS total_trip,
        ROUND(AVG(trip_distance_km), 2) AS avg_distance_km, 
        ROUND(AVG(total_amount), 2) AS average_amount_per_trip,
        ROUND(SUM(total_amount), 2) AS total_revenue
  FROM base
  GROUP BY color, PU_Borough, DR_Borough, y, m, ym, dow, hr
)

SELECT color,
      PU_Borough,
      DR_Borough,
      date_format(ym, 'yyyy-MM') AS year_month,
      CASE dow 
        WHEN 1 THEN 'Sunday'
        WHEN 2 THEN 'Monday'
        WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday'
        WHEN 5 THEN 'Thursday'
        WHEN 6 THEN 'Friday'
        WHEN 7 THEN 'Saturday'
      END AS dow,
      hr,
      total_trip,
      avg_distance_km, 
      average_amount_per_trip,
      total_revenue
FROM derived
ORDER BY color, PU_Borough, DR_Borough, year_month, dow, hr;

color,PU_Borough,DR_Borough,year_month,dow,hr,total_trip,avg_distance_km,average_amount_per_trip,total_revenue
green,Bronx,Bronx,2014-01,Friday,0,187,3.7,10.13,1894.98
green,Bronx,Bronx,2014-01,Friday,1,142,3.39,9.61,1365.1
green,Bronx,Bronx,2014-01,Friday,2,82,3.94,10.33,847.25
green,Bronx,Bronx,2014-01,Friday,3,62,3.46,9.82,608.56
green,Bronx,Bronx,2014-01,Friday,4,56,3.56,10.14,568.0
green,Bronx,Bronx,2014-01,Friday,5,68,3.67,9.81,667.0
green,Bronx,Bronx,2014-01,Friday,6,129,4.02,10.09,1301.1
green,Bronx,Bronx,2014-01,Friday,7,493,3.53,10.18,5017.1
green,Bronx,Bronx,2014-01,Friday,8,808,3.37,10.17,8220.29
green,Bronx,Bronx,2014-01,Friday,9,577,3.46,9.96,5745.19


## 2.4

In [0]:
%sql
WITH
base_2024 AS (
  SELECT PU_Borough,
        DR_Borough,
        total_amount
  FROM bde.final_trips_cleaned
  WHERE year(pickup_dt) = 2024
),

pair_rev AS (
  SELECT PU_Borough,
        DR_Borough,
        ROUND(SUM(total_amount), 2) AS pair_revenue
  FROM base_2024
  GROUP BY PU_Borough, DR_Borough
),

total_rev AS (
  SELECT SUM(pair_revenue) AS total_revenue
  FROM pair_rev
),

rank AS (
  SELECT p.PU_Borough,
        p.DR_Borough,
        p.pair_revenue,
        ROUND((p.pair_revenue / t.total_revenue) * 100, 2) AS total_rev_share_pct,
        ROW_NUMBER() OVER (ORDER BY p.pair_revenue DESC) as rnk
  FROM pair_rev p CROSS JOIN total_rev t
)

SELECT rnk AS rank,
      PU_Borough,
      DR_Borough,
      pair_revenue AS total_revenue_2024,
      total_rev_share_pct AS total_revenue_share_percentage_2024
FROM rank
WHERE rnk <=10
ORDER BY rnk ASC;


rank,PU_Borough,DR_Borough,total_revenue_2024,total_revenue_share_percentage_2024
1,Manhattan,Manhattan,7.1283499423E8,62.83
2,Queens,Manhattan,1.7379387119E8,15.32
3,Manhattan,Queens,7.453294027E7,6.57
4,Queens,Brooklyn,3.870417769E7,3.41
5,Manhattan,Brooklyn,3.292714195E7,2.9
6,Queens,Queens,3.292479983E7,2.9
7,Manhattan,EWR,1.253214581E7,1.1
8,Queens,Unknown,9722054.02,0.86
9,Brooklyn,Brooklyn,7811496.16,0.69
10,Brooklyn,Manhattan,7720958.5,0.68


## 2.5

In [0]:
%sql
SELECT trip_with_tips, total_trip,
      ROUND(trip_with_tips / total_trip *100, 2) AS pct_trip_with_tips
FROM (SELECT 
  (SELECT COUNT(*) FROM bde.final_trips_cleaned WHERE tip_amount > 0) AS trip_with_tips,
  (SELECT COUNT(*) FROM bde.final_trips_cleaned) AS total_trip
)


trip_with_tips,total_trip,pct_trip_with_tips
612049272,970338790,63.08


## 2.6

In [0]:
%sql
SELECT trip_with_tips, tip_15 AS trip_tips_15_or_more,
      ROUND(tip_15 / trip_with_tips *100, 2) AS pct_trip_with_tips_15_or_more
FROM (SELECT 
  (SELECT COUNT(*) FROM bde.final_trips_cleaned WHERE tip_amount > 0) AS trip_with_tips,
  (SELECT COUNT(*) FROM bde.final_trips_cleaned WHERE tip_amount >=15) AS tip_15
)

trip_with_tips,trip_tips_15_or_more,pct_trip_with_tips_15_or_more
612049272,4980039,0.81


## 2.7

In [0]:
%sql
WITH 
base AS (
  SELECT
    (unix_timestamp(dropoff_dt)-unix_timestamp(pickup_dt)) / 60 AS duration_min,
    trip_distance * 1.60934 AS trip_distance_km,
    total_amount
  FROM bde.final_trips_cleaned
  WHERE total_amount > 0                --fix part 1 later
),

derived_bin AS (
  SELECT 
    CASE 
      WHEN duration_min < 5 THEN '<5'
      WHEN duration_min < 10 THEN '5-10'
      WHEN duration_min < 20 THEN '10-20'
      WHEN duration_min < 30 THEN '20-30'
      WHEN duration_min < 60 THEN '30-60'
      ELSE '>60'
    END AS duration_bin,
    duration_min,
    trip_distance_km,
    (trip_distance_km / (duration_min/60)) AS speed_kmh,
    (trip_distance_km / total_amount) AS km_per_dollar
  FROM base
)

SELECT duration_bin,
      ROUND(AVG(speed_kmh), 2) AS avg_speed_kmh,
      ROUND(AVG(km_per_dollar), 2) AS avg_km_per_dollar
FROM derived_bin
GROUP BY duration_bin
ORDER BY CASE duration_bin
            WHEN '<5' THEN 1
            WHEN '5-10' THEN 2
            WHEN '10-20' THEN 3
            WHEN '20-30' THEN 4
            WHEN '30-60' THEN 5
            ELSE 6
          END;

duration_bin,avg_speed_kmh,avg_km_per_dollar
<5,20.03,0.17
5-10,17.22,0.21
10-20,17.92,0.26
20-30,21.39,0.31
30-60,25.8,0.38
>60,22.68,0.56
